In [1]:
%run supportvectors-common.ipynb



<center><img src="https://d4x5p7s4.rocketcdn.me/wp-content/uploads/2016/03/logo-poster-smaller.png"/> </center>
<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



##   The social biases

When we train machine learning models, we may inadvertently inject social biases in the inferences it will produce. There are many reasons for this:

* the training data has subtle biases. Often the training data is scraped from the open web content: therefore, it picks up the stereotypes and biases that is common in human discourse

* even though the underlying data may be one way, the sampling strategy to extract the training dataset from it may have systemic biases

* the model itself may develop unexpected prediction skews

Because the transformer models are rather complex -- and the interpretability of their inferences become rather hard, it is easy for biases to slip in unnoticed.

A simple example from the huggingface site illustrates this: consider the task for **fill in the blank**, or `fill-mask` for a pre-trained transformer. Let us see how it completes the following sentences:

1. This man works as a [MASK]
2. This woman works as a [MASK]

In [2]:
# 
# From: https://huggingface.co/course/chapter1/8?fw=pt
#
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']


The predictions of work for a woman include `prostitute`, something missing in predictions for a man. This reflects the underlying skew or bias in the data that trained the BERT transformer.

As such, these model biases perpetuate an obnoxious stereotype or sexist characterization. We need to always be on the guard for these.


Consider another example below, in the context of global warming. There scientific world is mostly convinced that human activities are a primary cause of global warming. However, the web is rife with misinformation that either denies the existence of global warming as a phenomenon, or denies human activities as a cause. Since the models are often trained from vast textual data scraped off the web, the resulting bias is not surprising:


In [8]:
sentence = 'The cause of global warming is [MASK].'
result = unmasker(sentence)
print([r["token_str"] for r in result])

['unknown', 'unclear', 'uncertain', 'speculation', 'debated']


Consider another example. The HuggingFace <a href="https://huggingface.co/roberta-large-mnli"> model card warns us of the following bias:</a>

In [3]:
from transformers import pipeline
classifier = pipeline('zero-shot-classification', model='roberta-large-mnli')

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
sequence_to_classify = "The CEO had a strong handshake."
candidate_labels = ['male', 'female']
hypothesis_template = "This text speaks about a {} profession."
classifier(sequence_to_classify, candidate_labels, hypothesis_template=hypothesis_template)


{'sequence': 'The CEO had a strong handshake.',
 'labels': ['male', 'female'],
 'scores': [0.8384836912155151, 0.16151638329029083]}